% Julia Scripts

In [2]:
using TSSOS
using DynamicPolynomials
using CSV
using Random
using Plots
using Statistics
using DataFrames

In [39]:
# read observations
Y=CSV.read("COMPASoutput.csv");
#Y=CSV.read("FairOutput.csv",header=0);

# set parameters
d=1
group = 2
traj = [2,2]
#traj = [2,3]
sumTraj = sum(traj);

In [32]:
function fairA(T,d,stp,enp,len)

# set nc operators
@ncpolyvar G Fdash m[1:T+1] q[1:T] z p[1:T] f[1:T]; # all varaibles are assumed to be nc
var=vcat(z,p,f,G,Fdash,m,q);

# constraints
ine1 = [f[i] - p[i] - Fdash*m[i+1] for i in 1:T];
ine2 = [- f[i] + p[i] + Fdash*m[i+1] for i in 1:T];
ine3 = [m[i+1] - q[i] - G*m[i] for i in 1:T];
ine4 = [- m[i+1] + q[i] + G*m[i] for i in 1:T];
max1 = [z-1/traj[1]*sum( 1/len[j]*sum((Y[t,j]-f[t])^2 for t in stp[j]:enp[j]) for j in 1:traj[1])];
max2 = [z-1/traj[2]*sum( 1/len[j]*sum((Y[t,j]-f[t])^2 for t in stp[j]:enp[j]) for j in traj[1]+1:sumTraj)];
#max3 = [z-(Y[t,j]-f[t])^2 for j in 1:sumTraj for t in stp[j]:enp[j]];

# objective
objF= - z;
#objU=-sum(sum((Y[t,j]-f[t])^2 for t in stp[j]:enp[j]) for j in 1:traj[1]);

# pop
popA=vcat(objF,ine1,ine2,ine3,ine4,max1,max2);
#popB=vcat(objF,ine1,ine2,ine3,ine4,com1,com2,com3,max3);
#popUnfair=vcat(objU,ine1,ine2,ine3,ine4,com1,com2,com3);

# solve model fair_A
optA,solA,dataA=blockcpop_first(popA,var,d,method="chordal"); #,method="chordal"
#optA,solA,dataA=blockcpop_higher!(dataA)

# solve model fair_B
#optB,solB,dataB=blockcpop_first(popB,var,d)
#optB,solB,dataB=blockcpop_higher!(dataB)

# solve model unfair
#optU,solU,dataU=blockcpop_first(popUnfair,var,d)
#optU,solU,dataU=blockcpop_higher!(dataU)
    
#return optA 
end

fairA (generic function with 1 method)

In [33]:
function fairB(T,d,stp,enp,len)

# set nc operators
@ncpolyvar G Fdash m[1:T+1] q[1:T] z p[1:T] f[1:T]; # all varaibles are assumed to be nc
var=vcat(z,p,f,G,Fdash,m,q);

# constraints
ine1 = [f[i] - p[i] - Fdash*m[i+1] for i in 1:T];
ine2 = [- f[i] + p[i] + Fdash*m[i+1] for i in 1:T];
ine3 = [m[i+1] - q[i] - G*m[i] for i in 1:T];
ine4 = [- m[i+1] + q[i] + G*m[i] for i in 1:T];
#max1 = [z-1/traj[1]*sum( 1/len[j]*sum((Y[t,j]-f[t])^2 for t in stp[j]:enp[j]) for j in 1:traj[1])];
#max2 = [z-1/traj[2]*sum( 1/len[j]*sum((Y[t,j]-f[t])^2 for t in stp[j]:enp[j]) for j in traj[1]+1:sumTraj)];
max3 = [z-(Y[t,j]-f[t])^2 for j in 1:sumTraj for t in stp[j]:enp[j]];

# objective
objF= - z;
#objU=-sum(sum((Y[t,j]-f[t])^2 for t in stp[j]:enp[j]) for j in 1:traj[1]);

# pop
#popA=vcat(objF,ine1,ine2,ine3,ine4,com1,com2,com3,max1,max2);
popB=vcat(objF,ine1,ine2,ine3,ine4,max3);
#popUnfair=vcat(objU,ine1,ine2,ine3,ine4,com1,com2,com3);

# solve model fair_A
#optA,solA,dataA=blockcpop_first(popA,var,d);
#optA,solA,dataA=blockcpop_higher!(dataA)

# solve model fair_B
optB,solB,dataB=blockcpop_first(popB,var,d,method="chordal") #,method="chordal"
#optB,solB,dataB=blockcpop_higher!(dataB)

# solve model unfair
#optU,solU,dataU=blockcpop_first(popUnfair,var,d)
#optU,solU,dataU=blockcpop_higher!(dataU)
    
#return optA 
end

fairB (generic function with 1 method)

In [40]:
Amean=Float64[]
Astd=Float64[]
Bmean=Float64[]
Bstd=Float64[]
for t in 5:30
    Am=[]
    Bm=[]
    for r in 1:3
        stp=vcat(1,rand(1:3,traj[1]-1),1,rand(1:2,traj[2]-1))
        enp=vcat(rand(t-2:t,traj[1]-1),t,rand(t-1:t,traj[2]-1),t)
        len=[enp[j]-stp[j]+1 for j in eachindex(stp)]
        Ae=@elapsed fairA(t,d,stp,enp,len);
        push!(Am,Ae)
        Be=@elapsed fairB(t,d,stp,enp,len);
        push!(Bm,Be)
    end
    push!(Amean,mean(copy(Am)))
    push!(Astd,std(copy(Am)))
    push!(Bmean,mean(copy(Bm)))
    push!(Bstd,std(copy(Bm)))
end

fblocksizes:
[2, 3, 4, 6]
[17, 1, 1, 1]
-----------------------------------------------
gblocksizes:
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]

Optimizer terminated. Time: 0.02    

termination status: SLOW_PROGRESS
solution status: UNKNOWN_RESULT_STATUS
optimum = -19373.14396671216
fblocksizes:
[2, 3, 4, 6]
[17, 1, 1, 1]
-----------------------------------------------
gblocksizes:
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------

gblocksizes:
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
---------------------------

24  2.2e-11  1.0e-01  1.2e-10  2.29e-01   -1.849386562e+04  -1.876047935e+04  7.2e-12  0.06  
25  2.1e-11  1.5e-01  1.1e-10  1.31e-01   -1.869542079e+04  -1.896070980e+04  6.9e-12  0.06  
26  2.1e-11  1.5e-01  1.1e-10  1.17e-01   -1.869542079e+04  -1.896070980e+04  6.9e-12  0.06  
27  2.1e-11  1.5e-01  1.1e-10  1.17e-01   -1.869542079e+04  -1.896070980e+04  6.9e-12  0.08  
Optimizer terminated. Time: 0.08    

termination status: SLOW_PROGRESS
solution status: UNKNOWN_RESULT_STATUS
optimum = -18695.420789830714
fblocksizes:
[2, 3, 4, 6]
[20, 1, 2, 1]
-----------------------------------------------
gblocksizes:
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
--------------------------------------

-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 72              
  Cones                  : 0               
  Scalar variables       : 46              
  Matrix variables       : 24              
  

...

-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------


optimum = -143601.56104816386
fblocksizes:
[2, 3, 4, 6]
[47, 1, 11, 1]
-----------------------------------------------
gblocksizes:
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
--------------------

-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------
[1]
[1]
-----------------------------------------------


Excessive output truncated after 524327 bytes.

[1]
[1]
-----------------------------------------------
[1]
[1]

In [41]:
A=convert(DataFrame,hcat(Amean,Astd))
CSV.write("compasAtime.csv",A, writeheader=false)
B=convert(DataFrame,hcat(Bmean,Bstd))
CSV.write("compasBtime.csv",B, writeheader=false)

#A=convert(DataFrame,hcat(Amean,Astd))
#CSV.write("FairAtime.csv",A, writeheader=false)
#B=convert(DataFrame,hcat(Bmean,Bstd))
#CSV.write("FairBtime.csv",B, writeheader=false)

"compasBtime.csv"